In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [6]:
from transformers import (
    BartModel, BartForConditionalGeneration, BartConfig, BartTokenizerFast,
    top_k_top_p_filtering
)
from transformers.models.bart.modeling_bart import shift_tokens_right


In [32]:
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-large')

In [33]:
seq2seq = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
seq2seq.config

BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "model_type": "bart",
  "no_repea

In [34]:
s = "Huggingface office is based in "

In [35]:
inputs = tokenizer(s, return_tensors='pt')
inputs

{'input_ids': tensor([[    0, 40710,  3923,  9021,   558,    16,   716,    11,  1437,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [36]:
input_ids = inputs['input_ids']
input_ids.size()

torch.Size([1, 10])

In [37]:
seq2seq_outputs = seq2seq(**inputs)
seq2seq_outputs.keys()

odict_keys(['logits', 'past_key_values', 'encoder_last_hidden_state'])

In [38]:
seq2seq_output_ids = torch.argmax(torch.softmax(seq2seq_outputs.logits, -1), -1)

In [39]:
tokenizer.decode(seq2seq_output_ids[0].tolist())

'HugHuggingface office is based in London�'

In [15]:
encoder = seq2seq.get_encoder()
decoder = seq2seq.get_decoder()

In [16]:
seq2seq.config

BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "model_type": "bart",
  "no_repeat_ng

In [17]:
encoder_outputs = encoder(**inputs)

In [18]:
encoder_outputs[0].size()

torch.Size([1, 10, 768])

In [19]:
decoder_input_ids = shift_tokens_right(
    input_ids, seq2seq.config.pad_token_id, seq2seq.config.decoder_start_token_id
)
bsz, seq_len = decoder_input_ids.size()
bsz, seq_len

(1, 10)

In [20]:
tokenizer.decode(decoder_input_ids[0].tolist())

'</s><s>Huggingface office is based in '

In [21]:
decoder_outputs = decoder(
    input_ids=decoder_input_ids,
    encoder_hidden_states=encoder_outputs[0]
)
decoder_outputs.keys()

odict_keys(['last_hidden_state', 'past_key_values'])

In [22]:
decoder_outputs['last_hidden_state'].size()

torch.Size([1, 10, 768])

In [23]:
decoder_outputs_logits = seq2seq.lm_head(decoder_outputs[0])

In [24]:
decoder_outputs_ids = torch.argmax(decoder_outputs_logits, -1)

In [25]:
tokenizer.decode(decoder_outputs_ids[0].tolist())

'<s>Huggingface office is based in </s>'

In [26]:
encoder_outputs[0].size()

torch.Size([1, 10, 768])

In [27]:
y = encoder_outputs[0]

In [28]:
y.view(1, -1).size()

torch.Size([1, 7680])

In [29]:
class IntoAdjMatrix(nn.Module):
    def __init__(self):
        super().__init__()
        self.dropout= nn.Dropout(p=0.3)
        self.fc_1 = nn.Linear(10240, 16)
        self.fc_2 = nn.Linear(16, 16)
        self.layer_norm = nn.LayerNorm(16)
        self.act_fn = nn.GELU()
        
    def forward(self, x):
        y = self.dropout(x)
        y = self.fc_1(y)
        res = y
        y = self.dropout(y)
        y = self.act_fn(y)
        y = self.fc_2(y)
        y = self.act_fn(y)
        y = res + y
        y = self.layer_norm(y)
        y = y.view(-1, 4, 4)
        return y

In [30]:
proj_to_adj_matrix = IntoAdjMatrix()

In [31]:
proj_to_adj_matrix(y.view(1, -1))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x7680 and 10240x16)

In [154]:
class IntoEntityEmbeddings(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin_1 = nn.Linear(10240, 400)
        
    def forward(self, x):
        y = self.lin_1(x)
        y = y.view(-1, 4, 100)
        return y

In [155]:
proj_to_entity_embeddings = IntoEntityEmbeddings()

In [156]:
proj_to_entity_embeddings(y.view(1, -1))

tensor([[[-4.6999e-02,  6.8538e-02,  1.5676e-02, -4.6299e-02, -1.6727e-02,
           8.9616e-02,  1.4776e-01, -6.1008e-04,  2.9207e-03,  1.9291e-02,
          -7.8850e-02, -6.0686e-02,  7.7052e-02,  4.6651e-02, -1.5538e-01,
          -3.2471e-02,  7.9341e-02,  9.7726e-02,  8.7774e-02, -2.8457e-02,
           3.1182e-02,  9.6361e-02,  2.2449e-02,  2.3355e-03, -2.9135e-02,
          -1.8153e-02,  5.4045e-02, -1.3971e-01,  5.4565e-03, -1.3574e-02,
           1.3266e-02,  9.4804e-02,  4.3338e-03, -4.9723e-02,  1.0478e-01,
          -4.6793e-03,  1.2139e-01, -1.3856e-02, -2.4786e-02, -5.0878e-03,
          -5.0077e-02,  1.2187e-01, -9.2251e-03, -2.7631e-02, -5.4822e-02,
           8.6438e-04, -1.3114e-02,  9.0280e-04, -1.0216e-01, -2.9567e-02,
          -5.5152e-02,  6.3946e-02,  3.9063e-02,  9.6491e-02, -7.3262e-03,
          -1.2501e-02,  2.0945e-02,  3.6878e-02,  4.3985e-02, -6.8632e-02,
           2.4357e-02, -5.9271e-02,  4.5543e-02,  1.1573e-01,  4.1075e-02,
          -6.6730e-02,  2